<a href="https://colab.research.google.com/github/SABRIS13/Introduction-to-Data-Science/blob/main/Big_Data_Pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <font color='CB1FA9'> Divide y Venceras: Big Data

## 1. Importando un archivo CSV gigante

“Generar un archivo CSV con únicamente los viajes que tuvieron ‘5’ estrellas”  El archivo tiene 150MM de líneas.

Chunksize

In [ ]:
# Definimos el tamaño máximo de filas de cada partición (1MM)
import pandas as pd
size = 1000000
# Creamos un objeto iterador 'df_chunk' que contendrá las particiones con 1MM de filas en cada iteración
df_chunk = pd.read_csv('rides.csv', chunksize=size)
# Creamos una variable booleana verdadera 'header' para exportar las cabeceras una única vez
header = True

# Ahora vamos a recorrer cada partición, realizar el filtro solicitado, y exportar el resultado a un nuevo CSV
# El atributo mode='a' (APPEND) sirve para no sobreescribir el archivo Resultado.csv en cada iteración, sino para siempre adjuntar los nuevos resultados
# Luego de la 1ra iteración 'header' vuelve a ser falsa para no colocar nuevamente las cabeceras en el CSV final
for chunk in df_chunk:
    chunk_filter = chunk[chunk['estrellas'] == 5]
    chunk_filter.to_csv('Resultado.csv', header=header, mode='a')
    header = False

## 2.  Importando una tabla SQL gigante

"Exportar a TXT el contenido de una tabla SQL “Factura” que contiene toda la facturación del mes de una conocida empresa de celulares"
Esta tabla tiene 100MM de líneas

Chunksize en conexiones sql

In [ ]:
# Importamos las bibliotecas
import pandas as pd
import teradatasql

# Configuramos los datos de conexión al banco SQL
user = "***"
password = "***"
host = '***'
query="SELECT * FROM FACTURA"

# Abrimos la conexión al Banco
with teradatasql.connect(host=host, user=user, password=password, logmech='LDAP') as connect:    
    # Creamos una variable booleana verdadera 'header' para exportar las cabeceras una única vez
    header = True
    # Ejecutamos la query de selección de la tabla 'Factura', pero importamos 1MM de filas de cada vez
    # Para cada 'chunk' (DataFrame con 1MM de filas) exportamos el resultado en un nuevo TXT utilizando el modo 'APPEND'
    for chunk in pd.read_sql(query, connect, chunksize=1000000):
        chunk.to_csv('Factura.txt', header=header, index=None, sep='|', mode='a')
        header = False

## 3.Importando múltiples archivos

“Subir estos 2 años de tráfico a una tabla y dividirla en archivos mensuales, donde cada archivo tendrá el tráfico de un mes”
Tenemos más de 200 archivos y además la suma de todos ellos generará 250MM

In [ ]:
# Importando bibliotecas
import pandas as pd
import glob

# Configurando el directorio donde se encuentran nuestros archivos
path = r'D:/files'
all_files = glob.glob(path + "/*")

# Ahora vamos a recorrer uno a uno los archivos de nuestro directorio, y dividir cada uno en archivos mensuales
for filename in all_files:
    # Cada archivo será importado al DataFrame 'datos', como no son archivos grandes no usaremos el atributo 'chunksize', aunque ya sabemos como usarlo
    datos = pd.read_csv(filename, sep = '|', header = None, dtype=str)
    # Crearemos nombres de columnas, porque los archivos no los tienen
    datos.columns = ['OPERATOR_ID','MONTH_DT','CUSTOMER_ID','SUBSCRIBER_ID','MSISDN_ID','IMEI_ID','IMSI_ID','TAC_ID','TRAFICO_DT','IMEI_IND','IMEI_ORIGIN_IND','USER_4P_ID','MSISDN_WITH_PREFIX_ID']
    # Actualizaremos el valor de la columna 'MONTH_DT' (YYYYMM) con el año-mes de la fecha del tráfico que se encuentra en la columna 'TRAFICO_DT' (YYYYMMDD)
    datos['MONTH_DT'] = datos['TRAFICO_DT'].str[0:6:1]
    # Ahora crearemos un objeto 'grupos' que tendrá una partición(DataFrame) por cada valor de la columna 'MONTH_DT' 
    grupos = datos.groupby('MONTH_DT')
    # Ahora que ya tenemos dividido nuestro archivo, vamos a guardar cada partición en su respectivo archivo mensual con el modo 'APPEND'
    for mes,valores in grupos:
        if mes=='201901':
            valores.to_csv(r'D:\resultados\TRAFICO_20190100', header=False, index=None, sep='|', mode='a')
        elif mes=='201902':
            valores.to_csv(r'D:\resultados\TRAFICO_20190200', header=False, index=None, sep='|', mode='a')
        # Completar para los demás meses del intervalo de 2 años
        elif mes=='202012':
            valores.to_csv(r'D:\resultados\TRAFICO_20201200', header=False, index=None, sep='|', mode='a')
        elif mes=='202101':
            valores.to_csv(r'D:\resultados\TRAFICO_20210100', header=False, index=None, sep='|', mode='a')
        else:
            valores.to_csv(r'D:\resultados\otros', header=False, index=None, sep='|', mode='a')